## Подход 1: градиентный бустинг "в лоб"

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [2]:
features = pd.read_csv('Data/features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
X_train = features.drop(['start_time', 'duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 
                         'barracks_status_radiant', 'barracks_status_dire'], axis = 1)
X_train.head()

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,7,11,5,2098,1489,20,0,0,7,67,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,0,42,4,1188,1033,9,0,1,12,49,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,7,33,4,1319,1270,22,0,0,12,98,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1,29,4,1779,1056,14,0,0,5,30,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,7,13,4,1431,1090,8,1,0,8,27,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [4]:
missing = (X_train.count() != X_train.shape[0])
missing_values = list(X_train.loc[:, missing].columns.values)

In [5]:
X_train = X_train.fillna(0)
y = features.radiant_win
y = y.fillna(0)

In [6]:
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

n_estimators = np.arange(10, 70, 10)

for n in n_estimators:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators = n)
    scores = cross_val_score(clf, X_train, np.ravel(y), cv = kf, scoring = 'roc_auc')
    end_time = datetime.datetime.now()
    total_time = end_time - start_time
    print('Number of estimators: %s, mean score: %s, total time: %s' % (n, scores.mean(), total_time))

Number of estimators: 10, mean score: 0.6648506879750012, total time: 0:01:53.936465
Number of estimators: 20, mean score: 0.6824618768044435, total time: 0:04:06.607309
Number of estimators: 30, mean score: 0.6900064710388155, total time: 0:06:09.378488
Number of estimators: 40, mean score: 0.6939415596721178, total time: 0:08:30.842636
Number of estimators: 50, mean score: 0.6974937440805975, total time: 0:12:13.969692
Number of estimators: 60, mean score: 0.7001763878925222, total time: 0:12:53.433075


##### 1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

In [7]:
missing_values

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

Пропуски в призаках могут быть из-за того, что некоторые события могли не наступить в первые 5 минут игры. 

Например, пропуск в признаке radiant_courier_time означает, что за первые 5 игровых минут команда не приобрела предмет "courier", а пропуск в признаке first_blood_player2 означает, что второй игрок не получил "first blood" за первые 5 минут игры.

##### 2. Как называется столбец, содержащий целевую переменную?

radiant_win

##### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти выше по тексту. Какое качество при этом получилось?

Время: 0:06:09.378488

Качество: 0.6900064710388155

##### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что можно сделать, чтобы ускорить его обучение при увеличении количества деревьев?

С увелечением количества деревьев увеличивается качество. 

Ускорить обучение можно, например, с помощью уменьшения размерности выборки. 

## Подход 2: логистическая регрессия

In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [9]:
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

n_estimators = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]

for n in n_estimators:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty = 'l2', C = n, max_iter = 150)
    scores = cross_val_score(clf, X_train_scaled, np.ravel(y), cv = kf, scoring = 'roc_auc')
    end_time = datetime.datetime.now()
    total_time = end_time - start_time
    print('C: %s, mean score: %s, total time: %s' % (n, scores.mean(), total_time))

C: 1e-06, mean score: 0.6881767139940977, total time: 0:00:02.770355
C: 1e-05, mean score: 0.6951197775458967, total time: 0:00:02.782832
C: 0.0001, mean score: 0.7112358063614078, total time: 0:00:03.211066
C: 0.001, mean score: 0.716243596719516, total time: 0:00:05.009220
C: 0.01, mean score: 0.7164355801469666, total time: 0:00:06.919583
C: 0.1, mean score: 0.7164095907941899, total time: 0:00:06.859510
C: 1, mean score: 0.7164069956808498, total time: 0:00:08.051969
C: 10, mean score: 0.7164075251713691, total time: 0:00:09.152026
C: 100, mean score: 0.7164074806994776, total time: 0:00:09.473649
C: 1000, mean score: 0.7164074912958517, total time: 0:00:09.250364


In [10]:
drop = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X_new_train = X_train.drop(drop, axis = 1)
X_new_train_scaled = scaler.fit_transform(X_new_train)

In [11]:
n_estimators = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]

for n in n_estimators:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty = 'l2', C = n, max_iter = 150)
    scores = cross_val_score(clf, X_new_train_scaled, np.ravel(y), cv = kf, scoring = 'roc_auc')
    end_time = datetime.datetime.now()
    total_time = end_time - start_time
    print('C: %s, mean score: %s, total time: %s' % (n, scores.mean(), total_time))

C: 1e-06, mean score: 0.6881313108412072, total time: 0:00:03.121006
C: 1e-05, mean score: 0.6950576560892063, total time: 0:00:02.767317
C: 0.0001, mean score: 0.7112109237766033, total time: 0:00:03.802024
C: 0.001, mean score: 0.7162552545858643, total time: 0:00:06.429532
C: 0.01, mean score: 0.7164426665509407, total time: 0:00:06.943619
C: 0.1, mean score: 0.7164162253687847, total time: 0:00:04.891148
C: 1, mean score: 0.7164131577919395, total time: 0:00:04.217612
C: 10, mean score: 0.7164126664186862, total time: 0:00:04.230842
C: 100, mean score: 0.7164123804461761, total time: 0:00:04.375715
C: 1000, mean score: 0.7164124164573022, total time: 0:00:04.454761


In [12]:
heroes = features[['r1_hero','r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                      'd1_hero','d2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
print('Unique heroes: ', np.shape(np.unique(heroes))[0])

Unique heroes:  108


In [13]:
n_heroes = 112
X_pick_train = np.zeros((X_train.shape[0], n_heroes))
for i, match_id in enumerate(X_train.index):
    for p in range(5):
        X_pick_train[i, X_train.loc[match_id, 'r%d_hero' % (p + 1)] - 1] = 1
        X_pick_train[i, X_train.loc[match_id, 'd%d_hero' % (p + 1)] - 1] = -1

X_in_bag = np.hstack((X_new_train_scaled, X_pick_train))

In [14]:
clf.fit(X_in_bag, y)
y_bag = clf.decision_function(X_in_bag)
print(roc_auc_score(y, y_bag))

0.7542959228381889


##### 1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Качество логистической регрессии: 0.7164355801469666 при С = 0.01. Качество градиентного бустинга 0.69. Такая разница может быть из-за недообучения градиентного бустинга. Логистическая регрессия работает гораздо быстрее градиентного бустинга (несколько секунд против нескольких минут). 

##### 2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

Качество совсем немного улучшилось: 0.7164355801469666 против 0.7164426665509407. Возможно, при большем количестве признаков происходит переобучение.

##### 3. Сколько различных идентификаторов героев существует в данной игре?

108

##### 4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

Качество при С = 0.01 улучшилось: 0.7164426665509407 против 0.7542959228381889. Улучшение качества связано с добавлением новых признаков. 

##### 5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [26]:
print('Min value: ', result.min())
print('Max value: ', result.max())

Min value:  radiant_win    0.008536
dtype: float64
Max value:  radiant_win    0.996673
dtype: float64


## Kaggle competition

Link: https://www.kaggle.com/c/dota-2-win-probability-prediction

In [16]:
X_test = pd.read_csv('Data/features_test.csv', index_col = 'match_id')
X_test = X_test.iloc[:,1:]
X_test.head()

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,0,93,4,1103,1089,8,0,1,9,102,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1,20,2,556,570,1,0,0,9,6,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1,112,2,751,808,1,0,0,13,26,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1,27,3,708,903,1,1,1,11,91,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1,39,4,1259,661,4,0,0,9,93,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [17]:
X_test = X_test.fillna(0)

In [18]:
X_pick_test = np.zeros((X_test.shape[0], n_heroes))
for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick_test[i, X_test.loc[match_id, 'r%d_hero' % (p + 1)] - 1] = 1
        X_pick_test[i, X_test.loc[match_id, 'd%d_hero' % (p + 1)] - 1] = -1

In [19]:
for i in drop:
    X_test.drop(i, inplace=True, axis=1)
X_scaled_test = scaler.fit_transform(X_test)

In [20]:
X_in_bag_test = np.hstack((X_scaled_test, X_pick_test))

In [21]:
y_test = clf.predict_proba(X_in_bag_test)

In [22]:
y_test = pd.DataFrame(y_test)
y_test.mean()

0    0.481795
1    0.518205
dtype: float64

Вероятность победы Radiant чуть выше.

In [23]:
result = y_test
result.columns = ['match_id', 'radiant_win']
result.index = X_test.index
result = result.drop(labels='match_id', axis=1)

In [24]:
result.head()

,radiant_win
match_id,
6,0.837138
7,0.779211
10,0.201093
13,0.869283
16,0.261494


In [25]:
result.to_csv('submission.csv', index = True)

Kaggle result: 0.75483